In [ ]:
!pip install accelerate
!pip install -U bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 7.1 MB/s eta 0:00:00


In [ ]:
import transformers
import torch
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import string
import re
import json
import os

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `New token` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `New toke

In [ ]:
!unzip partitions.zip -d /content/partitions

Archive:  partitions.zip
  inflating: /content/partitions/part_3.json  
  inflating: /content/partitions/part_4.json  
  inflating: /content/partitions/part_5.json  
  inflating: /content/partitions/part_6.json  
  inflating: /content/partitions/part_7.json  
  inflating: /content/partitions/part_8.json  
  inflating: /content/partitions/part_9.json  
  inflating: /content/partitions/part_10.json  
  inflating: /content/partitions/part_11.json  
  inflating: /content/partitions/part_12.json  
  inflating: /content/partitions/part_13.json  
  inflating: /content/partitions/part_14.json  
  inflating: /content/partitions/part_15.json  
  inflating: /content/partitions/part_16.json  
  inflating: /content/partitions/part_17.json  
  inflating: /content/partitions/part_18.json  
  inflating: /content/partitions/part_19.json  
  inflating: /content/partitions/part_20.json  
  inflating: /content/partitions/part_21.json  
  inflating: /content/partitions/part_22.json  
  inflating: /content/

In [ ]:
!mkdir processed_partitions

In [ ]:
model_id = "unsloth/llama-3-8b-Instruct-bnb-4bit"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
    model_kwargs={
        "torch_dtype": torch.float16,
        "quantization_config": {"load_in_4bit": True},
        "low_cpu_mem_usage": True,
    },
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/220 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

In [ ]:
def generate_summary(title, artist, lyrics, annotation):
    messages = [
        {"role": "system", "content": """Я хочу, чтобы ты прочитал текст песни и в трех или четырех коротких предложениях предложениях сказал о чём она.
        У песни может быть задана аннотация, если сочтешь её релевантной, можешь опираться на неё при генерации описания. Ответ генерируй на русском языке"""},
        {"role": "user", "content": f"Песня: {title} \n автор: {artist} Аннотация: {annotation} \n Текст песни: {lyrics}"}
    ]

    prompt = pipeline.tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    output = pipeline(
        prompt,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

    return output[0]["generated_text"][len(prompt):]

In [ ]:
def process_file(input_path, output_path):
    # Загрузка DataFrame
    ds = pd.read_json(input_path)

    # Проверка на наличие нужных столбцов
    required_columns = ["title", "artist", "lyrics", "annotation"]
    if not all(col in ds.columns for col in required_columns):
        print(f"Пропущен файл {input_path}: отсутствуют необходимые столбцы")
        return

    # Генерация суммаризаций
    summaries = []
    for index, row in tqdm(ds.iterrows(), desc=f"Processing {os.path.basename(input_path)}", total=ds.shape[0]):
        # Генерация суммаризации
        summary = generate_summary(row["title"], row["artist"], row["lyrics"], row["annotation"])
        summaries.append(summary)

    # Добавление нового столбца в DataFrame
    ds["summarization"] = summaries

    # Сохранение обработанного DataFrame
    ds.to_json(output_path, orient="records", force_ascii=False, indent=4)
    print(f"Файл обработан и сохранён: {output_path}")

In [ ]:
def generate_batch_summaries(batch):
    """
    Генерирует суммаризации для батча строк.
    """
    prompts = []
    for _, row in batch.iterrows():
        messages = [
            {"role": "system", "content": """Я хочу, чтобы ты прочитал текст песни и в трех или четырех коротких предложениях сказал о чём она.
            У песни может быть задана аннотация, если сочтешь её релевантной, можешь опираться на неё при генерации описания. Ответ генерируй на русском языке"""},
            {"role": "user", "content": f"Песня: {row['title']} \n автор: {row['artist']} Аннотация: {row['annotation']} \n Текст песни: {row['lyrics']}"}
        ]

        prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        prompts.append(prompt)

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    # Генерация текстов для всех запросов в батче
    outputs = pipeline(
        prompts,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
        return_full_text=False
    )

    # Возвращаем текст как есть, без попытки индексации по ключу
    return [output for output in outputs]


In [ ]:
def process_file_batch(input_path, output_path, batch_size=16):
    """
    Обработка файла с батчовой генерацией.
    """
    ds = pd.read_json(input_path)

    required_columns = ["title", "artist", "lyrics", "annotation"]
    if not all(col in ds.columns for col in required_columns):
        print(f"Пропущен файл {input_path}: отсутствуют необходимые столбцы")
        return

    summaries = []
    for i in tqdm(range(0, ds.shape[0], batch_size), desc=f"Processing {os.path.basename(input_path)}"):
        batch = ds.iloc[i:i + batch_size]
        batch_summaries = generate_batch_summaries(batch)
        summaries.extend(batch_summaries)

    ds["summarization"] = summaries

    ds.to_json(output_path, orient="records", force_ascii=False, indent=4)
    print(f"Файл обработан и сохранён: {output_path}")

In [ ]:
input_dir = "partitions"
output_dir = "processed_partitions"


In [ ]:
# for filename in os.listdir(input_dir):
#         input_path = os.path.join(input_dir, filename)
#         output_path = os.path.join(output_dir, filename)
#         if filename.endswith(".json"):
#             process_file(input_path, output_path)


Processing test.json: 100%|██████████| 2/2 [00:59<00:00, 29.79s/it]


Файл обработан и сохранён: processed_partitions/test.json


Processing test2.json: 100%|██████████| 2/2 [00:55<00:00, 27.87s/it]

Файл обработан и сохранён: processed_partitions/test2.json


In [ ]:
for filename in os.listdir(input_dir):
    input_path = os.path.join(input_dir, filename)
    output_path = os.path.join(output_dir, filename)
    if filename.endswith(".json"):
        process_file_batch(input_path, output_path, batch_size=16)


Processing part_1.json:   1%|          | 1/159 [06:36<17:24:13, 396.54s/it]


KeyboardInterrupt: 